In [1]:
import os
import sys
import time
import math
import numpy as np
import pandas as pd
import geopandas as gpd
folder = "C:\\Users\\celian\\OneDrive\\WorldBank\\Cambodia"

# Produce tech coef

In [5]:
techCoef = pd.read_excel(os.path.join(folder, "Data", "Structured", "InputOutputTable", "inputOutputTable.xlsx"), sheet_name="domestic_techCoef")
techCoef = techCoef.iloc[:-1,:]
techCoef.to_csv(os.path.join(folder, "Data", "Structured", "InputOutputTable", "techCoef.csv"))

# Produce export table

In [48]:
trade_data = pd.read_csv(os.path.join(folder, "Data", "Raw", "Trade", "comtrade_2015_CambodiaAsPartner_HS07_H3.csv"))

# because we chose "Cambodia as Partner", the "Trade Flow" (import/export) is to be understood from the point of view of the "Reporting Country"
export_data = trade_data.loc[trade_data['Trade Flow']=="Import", ['Reporter ISO', 'Commodity Code', 'Commodity', 'Trade Value (US$)']]

# groupby country region
country_grouping_modified = pd.read_csv(os.path.join(folder, "Data", "Raw", "Trade", "country_grouping_modified.csv"))
export_data = export_data.merge(country_grouping_modified[['alpha-3', "region"]], how="left", left_on="Reporter ISO", right_on="alpha-3")
export_data = export_data.groupby(['region', "Commodity Code"])['Trade Value (US$)'].sum().reset_index()

# map sectors
H3_to_Sector = pd.read_csv(os.path.join(folder, "Data", "Raw", "Trade", "H3_to_Sector.csv")).set_index('Commodity Code')['sector_code']
sector_to_trigram = pd.read_csv(os.path.join(folder, "Data", "Structured", "InputOutputTable", "sector_trigram.csv")).set_index('id')['trigram']
export_data['sector_code'] = export_data['Commodity Code'].map(H3_to_Sector)

print("Check unmapped sectors")
print(export_data[export_data["sector_code"].isnull()])
print("Represents "+"{:,.2f}%".format(100*export_data.loc[export_data["sector_code"].isnull(), 'Trade Value (US$)'].sum()/export_data['Trade Value (US$)'].sum()))
print("Skip them")
export_data = export_data[~export_data['sector_code'].isnull()]
export_data['sector_code'] = export_data['sector_code'].astype(int)
export_data['sector'] = export_data['sector_code'].map(sector_to_trigram)
print(export_data.head())
export_data = export_data.groupby(['region', 'sector'])['Trade Value (US$)'].sum().unstack(level=0)

# map country name
country_dic = pd.read_csv(os.path.join(folder, "Data", "Structured", "Trade", "country_table.csv")).set_index('country_name')['trigram'].to_dict()
country_dic['Americas'] = country_dic['America'] #there is an S in the orignal table
export_data = export_data.rename(columns=country_dic)
export_data = export_data[export_data.columns.sort_values()]

# get all sector, set to 0
all_sectors = pd.read_csv(os.path.join(folder, "Data", "Structured", "InputOutputTable", "sector_table.csv"))[['sector']]
export_data = export_data.reset_index().merge(all_sectors, how="right", on="sector")
export_data = export_data.set_index('sector').fillna(0)
export_data

# export
export_data.transpose().to_csv(os.path.join(folder, "Data", "Structured", "Trade", "export_table.csv"))

Check unmapped sectors
       region  Commodity Code  Trade Value (US$)  sector_code
49     Africa              99             121512          NaN
120  Americas              99            8180178          NaN
209      Asia              99            8708586          NaN
302    Europe              99            1846239          NaN
362   Oceania              99                530          NaN
499  Viet Nam              99               2456          NaN
Represents 0.13%
Skip them
   region  Commodity Code  Trade Value (US$)  sector_code sector
0  Africa               1                  7            2    LIV
1  Africa               9               2777            1    CRO
2  Africa              10             127529            1    CRO
3  Africa              12               2749            6    MFO
4  Africa              21              32969            6    MFO


# Produce import table

In [49]:
trade_data = pd.read_csv(os.path.join(folder, "Data", "Raw", "Trade", "comtrade_2015_CambodiaAsPartner_HS07_H3.csv"))

# because we chose "Cambodia as Partner", the "Trade Flow" (import/export) is to be understood from the point of view of the "Reporting Country"
import_data = trade_data.loc[trade_data['Trade Flow']=="Export", ['Reporter ISO', 'Commodity Code', 'Commodity', 'Trade Value (US$)']]

# groupby country region
country_grouping_modified = pd.read_csv(os.path.join(folder, "Data", "Raw", "Trade", "country_grouping_modified.csv"))
import_data = import_data.merge(country_grouping_modified[['alpha-3', "region"]], how="left", left_on="Reporter ISO", right_on="alpha-3")
import_data = import_data.groupby(['region', "Commodity Code"])['Trade Value (US$)'].sum().reset_index()

# map sectors
H3_to_Sector = pd.read_csv(os.path.join(folder, "Data", "Raw", "Trade", "H3_to_Sector.csv")).set_index('Commodity Code')['sector_code']
sector_to_trigram = pd.read_csv(os.path.join(folder, "Data", "Structured", "InputOutputTable", "sector_trigram.csv")).set_index('id')['trigram']
import_data['sector_code'] = import_data['Commodity Code'].map(H3_to_Sector)
print("Check unmapped sectors")
print(import_data[import_data["sector_code"].isnull()])
print("Represents "+"{:,.2f}%".format(100*import_data.loc[import_data["sector_code"].isnull(), 'Trade Value (US$)'].sum()/import_data['Trade Value (US$)'].sum()))
print("Skip them")
import_data = import_data[~import_data['sector_code'].isnull()]
import_data['sector_code'] = import_data['sector_code'].astype(int)
import_data['sector'] = import_data['sector_code'].map(sector_to_trigram)
import_data = import_data.groupby(['region', 'sector'])['Trade Value (US$)'].sum().unstack(level=0)

# map country name
country_dic = pd.read_csv(os.path.join(folder, "Data", "Structured", "Trade", "country_table.csv")).set_index('country_name')['trigram'].to_dict()
country_dic['Americas'] = country_dic['America'] #there is an S in the orignal table
import_data = import_data.rename(columns=country_dic)
import_data = import_data[import_data.columns.sort_values()]

# get all sector, set to 0
all_sectors = pd.read_csv(os.path.join(folder, "Data", "Structured", "InputOutputTable", "sector_table.csv"))[['sector']]
import_data = import_data.reset_index().merge(all_sectors, how="right", on="sector")
import_data = import_data.set_index('sector').fillna(0)
export_data

# export
import_data.transpose().to_csv(os.path.join(folder, "Data", "Structured", "Trade", "import_table.csv"))

Check unmapped sectors
       region  Commodity Code  Trade Value (US$)  sector_code
41     Africa              99              11944          NaN
124  Americas              99            3611327          NaN
221      Asia              99           18524435          NaN
312    Europe              99            4215140          NaN
385   Oceania              99            1633929          NaN
564  Viet Nam              99             243218          NaN
Represents 0.15%
Skip them


# Produce country importance as importer

In [88]:
# load data
sectorFromImportSector_techCoef = pd.read_excel(os.path.join(folder, "Data", "Structured", "InputOutputTable", "inputOutputTable.xlsx"), sheet_name="sectorImports_techCoef")
import_per_contry_sector_abs = pd.read_csv(os.path.join(folder, "Data", "Structured", "Trade", "import_table.csv")).set_index("sector_code").transpose()

# get relative importance of each country for each product type (=sector)
import_per_contry_sector_rel = import_per_contry_sector_abs/import_per_contry_sector_abs.sum()

# merge with sector for which no data to get full matrix
import_per_contry_sector_rel = import_per_contry_sector_rel.transpose().reset_index()
import_per_contry_sector_rel = import_per_contry_sector_rel.merge(pd.DataFrame({'sector_code':sectorFromImportSector_techCoef.index.tolist()}), how="right", on="sector_code")

# To fill the sector for which no country elements, we suppose that only Asia, Thailand, Viet Nam (because many services, suppose local, and these 3 represent over 95% anyway)
# And we suppose that it is in the same proportions between the three as for the other sectors
proportion = {}
selected_country = ['C5', 'C1', 'C0']
for country in selected_country:
    proportion[country] = import_per_contry_sector_rel[country].sum() / import_per_contry_sector_rel[selected_country].sum().sum()

boolindex_country_to_treat = import_per_contry_sector_rel.iloc[:,1:].isnull().all(axis=1)
for country in selected_country:
    import_per_contry_sector_rel.loc[boolindex_country_to_treat, country] = proportion[country]

# put the rest to 0
import_per_contry_sector_rel = import_per_contry_sector_rel.fillna(0)

# sort and prepare for matrix operation
import_per_contry_sector_rel = import_per_contry_sector_rel.sort_values('sector_code').set_index('sector_code')

# compute coef tech of country sector, i.e., how much of each domestic sector by from a foreign country to produce one unit of output
countrySector_coefTech = sectorFromImportSector_techCoef.transpose().dot(import_per_contry_sector_rel).transpose()

# this tells the importance of each country as importer for specific sectors
countrySector_importance = countrySector_coefTech / countrySector_coefTech.sum()
countrySector_importance = countrySector_importance.transpose()

# export
countrySector_importance.to_csv(os.path.join(folder, "Data", "Structured", "Trade", "countryImportanceForSector.csv"))

# Transit matrix

In [28]:
# Build matrix
countries = pd.read_csv(os.path.join(folder, "Data", "Structured", "Trade", "country_table.csv"))['trigram'].tolist()
transit_matrix = pd.DataFrame(index=countries, columns=countries, data=0)
transit_matrix

,VNM,THA,LAO,AFR,AME,ASI,EUR,OCE
VNM,0,0,0,0,0,0,0,0
THA,0,0,0,0,0,0,0,0
LAO,0,0,0,0,0,0,0,0
AFR,0,0,0,0,0,0,0,0
AME,0,0,0,0,0,0,0,0
ASI,0,0,0,0,0,0,0,0
EUR,0,0,0,0,0,0,0,0
OCE,0,0,0,0,0,0,0,0


In [34]:
# We assume that there are no transit flows, except between Viet Nam and Thailand
# Rationale is that VNM and THA have big ports, they do not need KHM to trade with other countries
# There seem to be almost nothing going on at the border between KHM and LAO
# A share of the trade between VNM and THA could go via the KHM terrestrial route, as opposed to port to port
# No data on that, so we assume 10%. Guesstimate.
VMM_THA = pd.read_csv(os.path.join(folder, "Data", "Raw", "Trade", "UNComTrade_ThaiilandVietnam.csv"))

VNM_to_THA = \
    (VMM_THA.loc[
        (VMM_THA['Trade Flow']=="Export")&\
        (VMM_THA['Reporter']=="Viet Nam")&\
        (VMM_THA['Partner']=="Thailand")
    ,'Trade Value (US$)'].iloc[0] +
     VMM_THA.loc[
        (VMM_THA['Trade Flow']=="Import")&\
        (VMM_THA['Reporter']=="Thailand")&\
        (VMM_THA['Partner']=="Viet Nam")
    ,'Trade Value (US$)'].iloc[0])/2
print("VNM_to_THA:", VNM_to_THA)

THA_to_VNM = \
    (VMM_THA.loc[
        (VMM_THA['Trade Flow']=="Export")&\
        (VMM_THA['Reporter']=="Thailand")&\
        (VMM_THA['Partner']=="Viet Nam")
    ,'Trade Value (US$)'].iloc[0] +
     VMM_THA.loc[
        (VMM_THA['Trade Flow']=="Import")&\
        (VMM_THA['Reporter']=="Viet Nam")&\
        (VMM_THA['Partner']=="Thailand")
    ,'Trade Value (US$)'].iloc[0])/2
print("THA_to_VNM:", THA_to_VNM)

VNM_to_THA: 5488260149.5
VNM_to_THA: 12500485144.5


In [37]:
HOW_MUCH_GOES_THROUGH_KHM = 0.1
transit_matrix.loc['VNM', 'THA'] = VNM_to_THA*HOW_MUCH_GOES_THROUGH_KHM
transit_matrix.loc['THA', 'VNM'] = THA_to_VNM*HOW_MUCH_GOES_THROUGH_KHM
transit_matrix

,VNM,THA,LAO,AFR,AME,ASI,EUR,OCE
VNM,0.000000e+00,5.488260e+08,0,0,0,0,0,0
THA,1.250049e+09,0.000000e+00,0,0,0,0,0,0
LAO,0.000000e+00,0.000000e+00,0,0,0,0,0,0
AFR,0.000000e+00,0.000000e+00,0,0,0,0,0,0
AME,0.000000e+00,0.000000e+00,0,0,0,0,0,0
ASI,0.000000e+00,0.000000e+00,0,0,0,0,0,0
EUR,0.000000e+00,0.000000e+00,0,0,0,0,0,0
OCE,0.000000e+00,0.000000e+00,0,0,0,0,0,0


In [38]:
transit_matrix.to_csv(os.path.join(folder, "Data", "Structured", "Trade", "transit_matrix.csv"), index=True)

### Export and import flows for analysis

In [16]:
rename_sec = {
    "CRO": "Agriculture",
    "LIV": "Livestock",
    "FOR": "Forestry & Logging",
    "FIS": "Fishing",
    "MIN": "Mining",
    "MFO": "Man. of Food Prod.",
    "MBE": "Man. of Beverages",
    "MTE": "Man. of Textiles",
    "MRU": "Man. of Rubber & Plastics",
    "ELE": "Utilities",
    "CON": "Construction",
    "TRW": "Wholesale Trade",
    "TRL": "Transport",
    "ACF": "Accomodation & Food Services",
    "TEL": "Telco",
    "FIN": "Financial Services",
    "REA": "Real Estate",
    "TRA": "Travel",
    "ADM": "Administration",
    "EDU": "Education",
    "HEA": "Health",
    "OPS": "Personnal Services",
    "MAO": "Other Man.",
    "MBA": "Man. of Basic Metals",
    "MCH": "Man. of Chemicals",
    "MCO": "Man. of Petroleum Prod.",
    "MEL": "Man. of Eletronics",
    "MFA": "Man. of Metal Prod.",
    "MFU": "Man. of Furniture",
    "MMA": "Man. of Machinery",
    "MPA": "Man. of Paper",
    "MPH": "Man. of Pharmaceuticals",
    "MTO": "Man. of Tobacco Prod.",
    "MTR": "Man. of Transport Eq.",
    "MWO": "Man. of Wood Prod.",
    "MNM": "Man. of Mineral Prod."
}
rename_country = {
    'AFR': "Africa",
    "AME": "America",
    "ASI": "Asia",
    "EUR": "Europe",
    "OCE": "Oceania",
    "THA": "Thailand",
    "VNM": "Viet Nam",
    "LAO": "Laos"
}

def renameCol(df, what, mapping):
    df[what+'_name'] = df[what].map(mapping)
    if df[what+'_name'].isnull().sum()>0:
        print("Unmapped values")
        print(df[df[what+'_name'].isnull()])
    return df

In [18]:
df = pd.read_csv(os.path.join(folder, "Data", "Structured", "Trade", "import_table.csv"), index_col=0)
df = df.unstack().reset_index()
df.columns = ['buying_sector', 'supplying_country', 'flow']
df = df[df['flow']>1000000]
df = renameCol(df, 'buying_sector', rename_sec)
df = renameCol(df, 'supplying_country', rename_country)
df.to_csv(os.path.join(folder, "Data", "Structured", "Trade", 'import_flows.csv'), index=False)

In [21]:
df = pd.read_csv(os.path.join(folder, "Data", "Structured", "Trade", "export_table.csv"), index_col=0)
df = df.unstack().reset_index()
df.columns = ['supplying_sector', 'buying_country', 'flow']
df = df[df['flow']>1000000]
df = renameCol(df, 'supplying_sector', rename_sec)
df = renameCol(df, 'buying_country', rename_country)
df.to_csv(os.path.join(folder, "Data", "Structured", "Trade", 'export_flows.csv'), index=False)